In [0]:
from pyspark.sql.functions import when, col, to_date
from pyspark.sql import functions as F
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
import seaborn as sns
import pandas as pd
import numpy as np
import math
import re


# 한글 글꼴 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우에서 일반적으로 지원되는 한글 글꼴 (Mac의 경우 'AppleGothic' 등 사용)
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호가 깨지는 문제 해결

## 1) 유의미한 샘플 수

In [0]:
# 통계적으로 유의한 샘플 크기 계산
def calculate_sample_size(df, confidence_level=0.95, margin_error=0.05):
    total_count = df.count()



    z_score = 2.576  # 99% 신뢰도
    p = 0.5  # 최대 분산
    
    n = (z_score**2 * p * (1-p)) / (margin_error**2)
    sample_size = n / (1 + (n-1)/total_count)  # n_adjusted
    print(f"필요 샘플 크기: {sample_size:,}")

    # 통계적 샘플 크기 계산
    sample_fraction = sample_size / total_count
    print(f"샘플링 비율: {sample_fraction:.7f}")

    return sample_fraction

## 2) 상관관계

### 상관관계 함수 - 개별


In [0]:
# 1. 층화 샘플링 함수 (기준년월별로 균등하게)
from pyspark.sql.functions import col

def stratified_sampling(df, sample_fraction, strata_col="기준년월"):
    """
    층화 샘플링으로 대표성 있는 샘플 생성 (seed 고정)
    """
    strata_samples = []
    
    for month in df.select(strata_col).distinct().collect():
        print("month content:", month)
        month_value = month[strata_col]
        month_df = df.filter(col(strata_col) == month_value)
        month_sample = month_df.sample(fraction=sample_fraction, seed=40)  # seed 고정
        strata_samples.append(month_sample)
    
    final_sample = strata_samples[0]
    for smp in strata_samples[1:]:  # ← 변수명 변경!
        final_sample = final_sample.union(smp)
    
    return final_sample

In [0]:
# 2. 빠른 상관관계 분석 (피쳐 수 제한 없음)
def fast_correlation_analysis(df, analysis_cols=None, method_option="pearson"):

    # 키 컬럼 제외
    exclude_cols = ['기준년월', '발급회원번호']
    # analysis_cols가 명시되지 않은 경우, 수치형 컬럼 자동 선택
    if analysis_cols is None:
        numeric_cols = [col_name for col_name, data_type in df.dtypes
                        if data_type in ['int', 'bigint', 'float', 'double']]
        analysis_cols = [col for col in numeric_cols if col not in exclude_cols]

    print(f"분석할 피처 수: {len(analysis_cols)}")
    
    # null 처리
    df_filled = df.fillna(0, subset=analysis_cols)
    
    # 기존 features 컬럼 제거
    if "features" in df_filled.columns:
        df_filled = df_filled.drop("features")

    # 벡터화
    assembler = VectorAssembler(inputCols=analysis_cols, outputCol="features")
    vector_df = assembler.transform(df_filled).select("features")
    
    # 캐싱
    vector_df.cache()
    vector_df.count()
    
    # 상관관계 계산
    print("상관관계 계산 중...")
    corr_matrix = Correlation.corr(vector_df, "features", method=method_option).head()[0]
    
    return corr_matrix, analysis_cols

In [0]:
# 3&4. 높은 상관관계 분석 (0.7 , 0.9이상)
def high_correlations(corr_matrix, cols_names, threshold=0.7):
    # 상관관계 매트릭스를 numpy 배열로 변환
    corr_array = corr_matrix.toArray()

    high_corrs = []
    multicollinear_pairs = []

    for i in range(len(cols_names)):
        for j in range(i + 1, len(cols_names)):
            corr_value = corr_array[i][j]


            if abs(corr_value) > 0.9: # 0.9 보다 큰
                multicollinear_pairs.append({
                    'feature1': cols_names[i],
                    'feature2': cols_names[j],
                    'correlation': corr_value
                })

            if abs(corr_value) > threshold: # 0.7 보다 큰
                high_corrs.append({
                    'feature1': cols_names[i],
                    'feature2': cols_names[j],
                    'correlation': corr_value
                })
    


    # 정렬 및 출력
    high_corrs_sorted = sorted(high_corrs, key=lambda x: abs(x['correlation']), reverse=True)

    print(f"\n=== 높은 상관관계 ({len(high_corrs_sorted)}개, threshold={threshold}) ===")
    for corr in high_corrs_sorted:
        if abs(corr_value) > 0.9: # 0.9 보다 큰
            print(f"⚠️ {corr['feature1']} ↔ {corr['feature2']}: {corr['correlation']:.3f}")
        print(f"{corr['feature1']} ↔ {corr['feature2']}: {corr['correlation']:.3f}")
    print(f"===== 끝 =====")
    return high_corrs_sorted, corr_array


In [0]:
# 5. 전체 및 높은 상관관계 시각화
def plot_correlation_heatmaps(corr_array, feature_names, threshold=0.7):
    """
    전체 상관관계 및 높은 상관관계 피처들만 시각화하는 히트맵 함수

    Parameters:
        corr_array (np.array): 상관관계 numpy 배열
        feature_names (List[str]): 피처 이름 리스트
        threshold (float): 상관관계 필터 기준값 (기본: 0.7)

    Returns:
        None
    """
    try:
        # 1. 전체 상관관계 히트맵
        corr_df = pd.DataFrame(corr_array, index=feature_names, columns=feature_names)
        print(f"전체 상관관계 행렬 크기: {corr_df.shape}")

        plt.figure(figsize=(20, 18))
        sns.heatmap(corr_df,
                    annot=False,
                    cmap='coolwarm',
                    center=0,
                    square=True,
                    fmt='.2f',
                    xticklabels=True,
                    yticklabels=True,
                    cbar_kws={'shrink': 0.8})
        plt.xticks(rotation=45, ha='right', fontsize=8)
        plt.yticks(rotation=0, fontsize=8)
        plt.title('Feature Correlation Heatmap (All Features)', fontsize=16)
        plt.tight_layout()
        plt.show()

        # 2. 높은 상관관계 피처들만 히트맵
        high_corr_features = set()
        for i in range(len(feature_names)):
            for j in range(i + 1, len(feature_names)):
                if abs(corr_array[i][j]) > threshold:
                    high_corr_features.add(feature_names[i])
                    high_corr_features.add(feature_names[j])

        if high_corr_features:
            high_corr_features = list(high_corr_features)
            print(f"\n=== 높은 상관관계 피처 수: {len(high_corr_features)}개 ===")

            corr_subset = corr_df.loc[high_corr_features, high_corr_features]

            plt.figure(figsize=(12, 10))
            sns.heatmap(corr_subset,
                        annot=False,
                        cmap='coolwarm',
                        center=0,
                        square=True,
                        fmt='.2f',
                        xticklabels=True,
                        yticklabels=True)
            plt.xticks(rotation=45, ha='right')
            plt.yticks(rotation=0)
            plt.title(f'High Correlation Features Heatmap (>|{threshold}|)')
            plt.tight_layout()
            plt.show()
        else:
            print("높은 상관관계를 가진 피처가 없습니다.")

    except ImportError:
        print("matplotlib/seaborn이 없어 히트맵을 생성할 수 없습니다.")
    except Exception as e:
        print(f"히트맵 생성 중 오류: {str(e)}")

### 상관관계 함수 - 전체

In [0]:
### 전체 한번에 함수
def correlation_func(df):
    # 통계적으로 유의한 샘플 크기 계산
    sample_fraction = calculate_sample_size(df)


    ### 1. 층화 샘플링 실행
    print("=== 층화 샘플링 실행 ===")
    sampled_df = stratified_sampling(df, sample_fraction)
    sampled_count = sampled_df.count()
    print(f"샘플 데이터: {sampled_count:,}")

    ### 2. 빠른 분석 실행 (모든 피처 사용)
    corr_matrix, cols_names = fast_correlation_analysis(sampled_df)
    print("상관관계 계산 완료!")

    ### 3. 높은 상관관계
    high_corrs, corr_array = high_correlations(corr_matrix, cols_names) # high_corrs_sorted, corr_array

    return corr_matrix, corr_array, cols_names


### 상관관계 - 히트맵

In [0]:
# 5. 전체 및 높은 상관관계 시각화
def plot_correlation_heatmaps(corr_array, feature_names, threshold=0.7):
    """
    전체 상관관계 및 높은 상관관계 피처들만 시각화하는 히트맵 함수

    Parameters:
        corr_array (np.array): 상관관계 numpy 배열
        feature_names (List[str]): 피처 이름 리스트
        threshold (float): 상관관계 필터 기준값 (기본: 0.7)

    Returns:
        None
    """
    try:
        # 1. 전체 상관관계 히트맵
        corr_df = pd.DataFrame(corr_array, index=feature_names, columns=feature_names)
        print(f"전체 상관관계 행렬 크기: {corr_df.shape}")

        plt.figure(figsize=(20, 18))
        sns.heatmap(corr_df,
                    annot=False,
                    cmap='coolwarm',
                    center=0,
                    square=True,
                    fmt='.2f',
                    xticklabels=True,
                    yticklabels=True,
                    cbar_kws={'shrink': 0.8})
        plt.xticks(rotation=45, ha='right', fontsize=8)
        plt.yticks(rotation=0, fontsize=8)
        plt.title('Feature Correlation Heatmap (All Features)', fontsize=16)
        plt.tight_layout()
        plt.show()

        # 2. 높은 상관관계 피처들만 히트맵
        high_corr_features = set()
        for i in range(len(feature_names)):
            for j in range(i + 1, len(feature_names)):
                if abs(corr_array[i][j]) > threshold:
                    high_corr_features.add(feature_names[i])
                    high_corr_features.add(feature_names[j])

        if high_corr_features:
            high_corr_features = list(high_corr_features)
            print(f"\n=== 높은 상관관계 피처 수: {len(high_corr_features)}개 ===")

            corr_subset = corr_df.loc[high_corr_features, high_corr_features]

            plt.figure(figsize=(12, 10))
            sns.heatmap(corr_subset,
                        annot=False,
                        cmap='coolwarm',
                        center=0,
                        square=True,
                        fmt='.2f',
                        xticklabels=True,
                        yticklabels=True)
            plt.xticks(rotation=45, ha='right')
            plt.yticks(rotation=0)
            plt.title(f'High Correlation Features Heatmap (>|{threshold}|)')
            plt.tight_layout()
            plt.show()
        else:
            print("높은 상관관계를 가진 피처가 없습니다.")

    except ImportError:
        print("matplotlib/seaborn이 없어 히트맵을 생성할 수 없습니다.")
    except Exception as e:
        print(f"히트맵 생성 중 오류: {str(e)}")

In [0]:
# plot_correlation_heatmaps(corr_array=ap_corr_array,
#                           feature_names=ap_cols_names)


### 상관계수 = ±1 제거

In [0]:
import numpy as np

def remove_correlated_features(corr_matrix, col_names):
    """
    상관계수 절댓값이 1.0인 컬럼쌍 중 하나를 제거
    """
    to_remove = set()
    n = len(col_names)
    
    for i in range(n):
        for j in range(i+1, n):
            if abs(corr_matrix[i, j]) >= 0.97:
                # 중복된 하나만 제거 (보통 후행 인덱스)
                to_remove.add(col_names[j])
    
    print(f"완전히 중복된 변수 수: {len(to_remove)}개")
    return [col for col in col_names if col not in to_remove]


# 테스트

In [0]:
# 테스트
if __name__ == "__main__":
    df_sample = spark.read.format("delta").table("database_pjt.3_use_encoding_sample")
    print(f"데이터 크기: {df_sample.count()}개 행, {len(df_sample.columns)}개 컬럼")
    display(df_sample.head(3))

    calculate_sample_size(df_sample)

In [0]:
if __name__ == "__main__":
    corr_matrix, corr_array, cols_names = correlation_func(df_sample)
    print(cols_names)